In [ ]:
import json

import fsspec
import geopandas
from shapely.geometry import Point

from carbonplan_forest_offsets_fires import utils

In [ ]:
fire_potential_months = {1: 'june', 2: 'july', 3: 'august', 4: 'september'}
ea_opr_ids = utils.list_all_ea_opr_ids()

In [ ]:
with fsspec.open('gcs://carbonplan-forest-offsets/web/display-data.json', 'r') as f:
    d = json.load(f)

gs = geopandas.GeoSeries(
    {
        project['opr_id']: Point(*project['shape_centroid'])
        for project in d
        if project['opr_id'] not in ea_opr_ids
    }
)

arbocs = {project['opr_id']: project['arbocs'] for project in d}

gdf = geopandas.GeoDataFrame({'id': gs.index}, geometry=gs.geometry.values)
gdf = gdf.set_crs('epsg:4326')

In [ ]:
store = {}
for num, month in fire_potential_months.items():
    risk = geopandas.read_file(
        f'/tmp/nifc-forecast/FirePotentialbyPSA_Month{num}.shp'
    )  # https://fsapps.nwcg.gov/psp/npsg/data/dynamic/monthly_extended_outlook.zip
    above = risk[risk['FirePotent'] == 'Above']

    projects_above_risk = geopandas.sjoin(above, gdf)['id'].unique().tolist()

    store[month] = round(sum([arbocs[opr_id] for opr_id in projects_above_risk]) / 1_000_000, 2)

with fsspec.open('gs://carbonplan-forest-offsets/fires/fire-season-2022-outlook.json', 'w') as f:
    json.dump(store, f)